# Assignment 5 - RNN
## Task: 4
Author: Soham Vaishnav
Roll No.: 2022112002

## Importing modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb

import os
import sys 
import shutil
import time

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

AssignDIR = os.path.dirname(os.path.dirname(os.path.abspath('RNN.ipynb')))
CurrDIR = os.path.dirname(os.path.abspath('RNN.ipynb'))
UserDIR = os.path.dirname(AssignDIR)

sys.path.append(UserDIR)

from sklearn.preprocessing import StandardScaler
import librosa

RawDataDIR = os.path.join(UserDIR, "./data/external/")
PreProcessDIR = os.path.join(UserDIR, "./data/interim/5/")

## Loading/Creating Dataset

In [12]:
def getData(data_length, max_seq_length=16):
    data= []
    np.random.seed(0)
    for i in range(data_length):
        seq_length = np.random.choice(list(range(1, max_seq_length + 1)))
        seq = np.random.choice([0, 1], size=(seq_length, 1)).tolist()
        if (len(seq) < max_seq_length):
            for i in range(max_seq_length - len(seq)):
                seq.append([0])
        data.append([seq, np.sum(seq)])
    return data

data_seq = getData(100000)

In [3]:
for i in range(10):
    print(data_seq[i])

[[[1], [1], [0], [1], [1], [1], [1], [1], [1], [1], [0], [0], [1], [0], [0], [0]], 10]
[[[0], [0], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0]], 2]
[[[0], [0], [1], [1], [1], [1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0]], 5]
[[[1], [0], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], 3]
[[[1], [0], [0], [1], [0], [1], [1], [1], [1], [1], [0], [1], [0], [1], [1], [1]], 11]
[[[0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [0], [0], [0], [0], [0], [1]], 6]
[[[0], [0], [0], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], 2]
[[[0], [0], [1], [0], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0]], 7]
[[[1], [1], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], 3]
[[[1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], 2]


## RNN Model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

In [5]:
train_data = data_seq[:int(len(data_seq)*0.8)]
val_data = data_seq[int(len(data_seq)*0.8):int(len(data_seq)*0.9)]
test_data = data_seq[int(len(data_seq)*0.9):]

class BitsData(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0], dtype=torch.float32), torch.tensor(self.data[idx][1], dtype=torch.float32)
    
train_loader = DataLoader(BitsData(train_data), batch_size=64, shuffle=True)
val_loader = DataLoader(BitsData(val_data), batch_size=64, shuffle=True)
test_loader = DataLoader(BitsData(test_data), batch_size=64, shuffle=True)

In [19]:
config = {
    'lr': 0.001,
    'epochs': 10,
    'hidden_size': 64,
    'n_layers': 2,
    'dropout': 0.2,
    'batch_norm': False,
    'input_size': 1,
    'nonlinearity': 'tanh'
}

In [20]:
from models.rnn.rnn import BitCounterRNN

model = BitCounterRNN(config)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
model.fit(train_loader=train_loader, val_loader=val_loader, 
          criterion=criterion, optimizer=optimizer, lr=config['lr'], n_epochs=config['epochs'])

Epoch 1/10, MAE Train Loss: 2.111297369003296, Val Loss: 2.2735445499420166
Epoch 2/10, MAE Train Loss: 1.9997220039367676, Val Loss: 2.263935089111328
Epoch 3/10, MAE Train Loss: 0.14738290011882782, Val Loss: 0.19880542159080505
Epoch 4/10, MAE Train Loss: 0.10613097250461578, Val Loss: 0.060667090117931366
Epoch 5/10, MAE Train Loss: 0.043768931180238724, Val Loss: 0.04025984928011894
Epoch 6/10, MAE Train Loss: 0.024079151451587677, Val Loss: 0.0335458368062973
Epoch 7/10, MAE Train Loss: 0.019223755225539207, Val Loss: 0.018636096268892288
Epoch 8/10, MAE Train Loss: 0.014206674881279469, Val Loss: 0.017037194222211838
Epoch 9/10, MAE Train Loss: 0.005224738270044327, Val Loss: 0.010067938826978207
Epoch 10/10, MAE Train Loss: 0.0031696560326963663, Val Loss: 0.017245927825570107
Model trained successfully


In [26]:
model.save_model(os.path.join(CurrDIR, 'RNN_best_model_1.pth')) 

Model saved successfully at  c:\Users\Admin\smai-m24-assignments-SohamVaishnav\assignments\5\RNN_best_model_1.pth


## Testing generalisation

In [27]:
generalisation_data_17 = getData(1000, 17)
generalisation_data_18 = getData(1000, 18)

generalisation_loader_17 = DataLoader(BitsData(generalisation_data_17), batch_size=64, shuffle=True)
generalisation_loader_18 = DataLoader(BitsData(generalisation_data_18), batch_size=64, shuffle=True)

In [28]:
loss_17 = model.evaluate(generalisation_loader_17, criterion)
loss_18 = model.evaluate(generalisation_loader_18, criterion)

print(f"Loss on 17-length sequences: {loss_17}")
print(f"Loss on 18-length sequences: {loss_18}")

Loss on 17-length sequences: 0.7939453125
Loss on 18-length sequences: 1.0788984298706055


## Loss vs Length of Sequence

In [33]:
losses_seq = []
for i in range(1, 33):
    generalisation_data = getData(1000, i)
    generalisation_loader = DataLoader(BitsData(generalisation_data), batch_size=64, shuffle=True)
    loss = model.evaluate(generalisation_loader, criterion)
    losses_seq.append(loss)

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, 33)), y=losses_seq, mode='lines+markers'))
fig.update_layout(title='Loss on sequences of different lengths',
                   xaxis_title='Sequence length',
                   yaxis_title='L1 loss')
fig.show()